# Model Prediction

In [2]:
import csv
import numpy as np
from datetime import datetime, timedelta
from lstm import Machine
import pandas as pd
import tensorflow as tf
from plotly.offline import init_notebook_mode
import cufflinks as cf

MODEL_PATH = "data/lstm.hd5"
CSV_PATH = "data/binance-BTCUSD.csv"
TRAIN_TEST_RATE = 0.85

cf.go_offline()
init_notebook_mode(connected=True)

with open(CSV_PATH) as fin:
    reader = [dict(item) for item in csv.DictReader(fin)]
machine = Machine(reader)
(_, _, X_test, Y_test) = machine.split_data(TRAIN_TEST_RATE)

times = [int(item['openTime']) for item in reader]
times = times[int(len(times) * TRAIN_TEST_RATE) + 4:]
times = [datetime.utcfromtimestamp(t / 1000.0) for t in times]
times_pred = times + [times[-1] + timedelta(days=n) for n in range(1, 401)]

model = tf.keras.models.load_model(MODEL_PATH)
pred = model.predict(X_test)

p = pred[-1]
x = X_test[-1]
x = np.delete(x, 0, 0)
x = np.append(x, [p], 0)

for c in range(400):
    p = model.predict(np.array([x]))
    pred = np.append(pred, p, 0)

    x = np.delete(x, 0, 0)
    x = np.append(x, p, 0)

pred = machine.scaler.inverse_transform(pred)
Y_test = machine.scaler.inverse_transform(Y_test)

for (c, key) in enumerate(['open', 'close', 'high', 'low']):
    pred_df = pd.DataFrame({
        'Predictions': [item[c] for item in pred],
    }, index=times_pred)
    act_df = pd.DataFrame({
        'Actual': [item[c] for item in Y_test]
    }, index=times)
    pd.concat([pred_df, act_df], axis=1).iplot(
        asFigure=False,
        kind='scatter',
        xTitle='Date',
        yTitle='Price',
        title=f'Price Prediction ({key})'
    )